# Twitter US Airline Sentiment

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10,6)
pd.set_option('display.max_columns', 50)

In [2]:
df = pd.read_csv('dataset/Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [5]:
df['text'].head()

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
Name: text, dtype: object

In [6]:
df.airline.value_counts()

airline
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [8]:
df['negativereason'].value_counts()

negativereason
Customer Service Issue         2910
Late Flight                    1665
Can't Tell                     1190
Cancelled Flight                847
Lost Luggage                    724
Bad Flight                      580
Flight Booking Problems         529
Flight Attendant Complaints     481
longlines                       178
Damaged Luggage                  74
Name: count, dtype: int64

In [ ]:
# 1- bu notebook'ta önce datayi oku ve gerekli featuresleri al
# 2- Datayi tanimak icin görsellestirmeler yap  
# 3- stopwords'larden n't ve not kelimeleri cikariliyor, cünkü bize lazim olacaklar
# 4- cleaning fonksiyonu ile temizlik yapilacak,http:wwww,@,#, bu isaretler temizlenecek, lower yapilacak, stopword uygulanacak,cleaning fonksiyonu apply(cleaning) ile uygulanacak
# 5- Train test split yapiliyor
    # X = df2['text']
    # y = df2['airline_sentiment']
# 6- istersen Count vectorization ile dataya vector halina cevir, burda kelimenin metinde gecen tüm kelimeleri unique olarak bir features tablosu yapiyor
# ve alacagi cümledeki kelimeler nelerde onlarin karsilarina gelen yerlere get_dummies gibi 0,1 yaziyor
# istersende asagida yaptigi gibi TF_IDF yapabiiriz, burda ise kelimenin cümle icindeki oranini bulup ve diger cümlelerde de geciyormu diye bakip ordaki oranida bulup carpiyor
# kendi icindeki Tf, baska cümlelerdeki de IDF oluyor
# bu ikisinden birini yapmalisin bunlar birbirinin alternatifi, ama TF IDf daha iyi duruyor gibi

# 7- en sonda modellere koyduktan sonra grafik cizerek hangi modelin daha iyi oldugunu görüyoruz
# 8- ve en iyi modeli sectikten sonra onunla yeni gelen twettleri predict yapabiliriz
# 9- istersen en sonda Word cloud olarak genel cerceveyi görebiliriz